- financedatareader에서 KRX(한국 종목)의 특정종목의 특정일 가격정보를 불러온다
- Name이 주식시장 종목명과 일치하지 않기 때문에 Symbol(종목코드)로 접근하여 총 10개 종목의 데이터를 수집한다.
- 당일 OPEN CLOSE의 변화를 나타낸 Change 값을 라벨링할 것이므로, Change가 0보다 크면 1(상승), 0보다 작으면 0(하락)으로 labeling 진행 

In [1]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np

In [2]:
# 기업정보를 나타내는 raw
raw = fdr.StockListing('KRX')
raw

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,박대현,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도
...,...,...,...,...,...,...,...,...,...,...
6896,000547,KOSPI,흥국화재2우B,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6897,000545,KOSPI,흥국화재우,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6898,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),1976-06-29,12월,이환구,http://www.heung-a.com,서울특별시
6899,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,2000-02-03,12월,"정영균, 이목운, 허철호 (각자대표)",http://www.heerim.com,서울특별시


In [3]:
# 크래프톤의 기업정보를 확인하는 예시
raw[raw['Symbol'] == '259960']

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
4237,259960,KOSPI,KRAFTON,소프트웨어 개발 및 공급업,등 게임 소프트웨어,2021-08-10,12월,김창한,http://www.krafton.com,서울특별시


In [3]:
# 크래프톤 예시, 기준일을 2018년부터로 설정해도 첫 상장일이 그 이후라면 첫 상장일부터 가격 정보를 얻을 수 있다. 
fdr.DataReader('259960', '2018')

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-08-10,448500,480000,400500,454000,5121520,NaN
2021-08-11,444500,446000,405000,407000,1647759,-0.103524
2021-08-12,414000,420500,402000,406000,947958,-0.002457
2021-08-13,415000,445500,408500,437000,1669847,0.076355
2021-08-17,433000,460000,423000,451500,1189500,0.033181
...,...,...,...,...,...,...
2022-01-13,364000,365000,345000,346500,570811,-0.057143
2022-01-14,338500,354000,333500,343500,462339,-0.008658
2022-01-17,351500,354500,338000,341000,298123,-0.007278


## 가격 데이터- 국내주식
- '단축코드'와 '연도' 사용

## 종목 선정 10개 기준은 2021년 시가총액이 큰 기업 중 계열사 제외 총 10개의 상위 기업으로 선정
- 가능한 모든 일자에 기사가 필요하므로 crawling할 헤드라인 개수가 일일 단위로 5개 이상이 되도록 상위 기업으로 선정

- 총 선정 기업 10개의 종목코드

삼성전자 005930  
SK하이닉스 000660  
NAVER 035420  
LG화학 051910  
현대차 005380    

카카오 035720  
셀트리온 068270  
POSCO 005490  
KB금융 105560  
신한지주 055550

In [18]:
# 가격데이터 한 데이터 프레임으로 정리하기
def make_df(dict_one):
    code, name = dict_one
    df = fdr.DataReader(code, '2012') # 2012.01.01 - 현재시점의 특정종목 주식시장 정보 
    df.reset_index(inplace=True)
    df['Change'][df['Change'] > 0] = 1
    df['Change'][df['Change'] < 0] = 0
    df['name'] = name
    df['Change'] = df['Change'].astype('int')
    return df

name = ['삼성전자','SK하이닉스','NAVER','LG화학','현대차','카카오','셀트리온','POSCO','KB금융','신한지주']
code = ['005930','000660','035420','051910','005380','035720','068270','005490','105560','055550']

d = dict(zip(code,name))
total = pd.DataFrame()
for dict_one in d.items():
    temp = make_df(dict_one)
    total = pd.concat([total,temp])


C:\Users\jhy7547\AppData\Local\Temp/ipykernel_17228/101421038.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Change'][df['Change'] < 0] = 0


In [19]:
cols = ['Date','Change','name']
total = total[cols]
# column 명 모두 소문자로 통일
total.rename(columns={'Date':'date','Change':'change'},inplace=True)

total.reset_index(inplace=True)
del total['index']
total

,date,change,name
0,2012-01-02,1,삼성전자
1,2012-01-03,1,삼성전자
2,2012-01-04,0,삼성전자
3,2012-01-05,0,삼성전자
4,2012-01-06,0,삼성전자
...,...,...,...
24725,2022-01-10,1,신한지주
24726,2022-01-11,1,신한지주
24727,2022-01-12,1,신한지주
24728,2022-01-13,0,신한지주


In [20]:
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24730 entries, 0 to 24729
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    24730 non-null  datetime64[ns]
 1   change  24730 non-null  int32         
 2   name    24730 non-null  object        
dtypes: datetime64[ns](1), int32(1), object(1)
memory usage: 483.1+ KB


In [22]:
total.to_csv('./ten_change.csv',encoding='utf-8')